# Merging some data

The datasets being merged only applies to california but if you can find some other datasets that have the name of the cities as a column it should not be too difficult since python and pandas provide convenient ways to work with [text](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html). 

In this notebook i will be merging the `latitude` and `longitude` variables from the kaggle dataset cal_cities_lat_long.csv. A `population` variable for each city after scraping it from towncharts.com demographics web page for each city. And finally, `area_total_km2` variable from Jake Vanderplas hosted california_cities.csv dat. Making it 4 new variable being merged funny enough i notice these last dataset had lat and long

In [2]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle

In [28]:
cities_data=pd.read_csv("C:\\Users\\Crist\\Towncharts\\California_Housing_Project\\datasets\\cal_cities_lat_long.csv")

In [26]:
housing = pd.read_csv("C:\\Users\\Crist\\Towncharts\\California_Housing_Project//datasets/Towncharts_California_Housing.csv",index_col =0)

Merging the `latitude` and `longitude`



The following lines of code will create a new city name column in our housing DataFrame that will be used to merge the appropriate latitude an longtitude to the correct row of the housing DataFrame. As currently our housing has the name of the city formatted as Alameda city-CA and the cal_cities_lat_long.csv  has the name formatted as Alemeda. 

In [20]:
name_of_city =  []

for indx,city in enumerate(housing.city_name):
    ser_city = pd.Series(city)
    before_len = len(name_of_city)
    for aname in cities_data.Name:
        if ser_city[ser_city.str.contains(aname)].empty == False:
            name_of_city.append(aname)
            break
            
    after_len = len(name_of_city)
    if after_len == before_len:
        name_of_city.append(None)
housing['city'] = name_of_city 

In [21]:
california_housing = housing.merge(cities_data, left_on='city',right_on='Name')

In [22]:
california_housing = california_housing.drop(columns = ['city_name','Name'])

Merging the `population` variable



After some reflecting I really wanted the population of each 
city so i chose to gather said variable

In [ ]:
population_links = {}
for city in cities_data.Name:
    population_links[city] = 'http://www.towncharts.com/California/Demographics/{}-city-CA-Demographics-data.html'.format(city)
population_dict = {}
for key,value in population_links.items():
    ua = UserAgent()
    t0 = time.time()
    response = requests.get(value,{"User-Agent": ua.random})
    
    if response.status_code != 200:
        continue
    response_delay = time.time() - t0
    time.sleep(10*response_delay)
    soup = BeautifulSoup(response.text,'html.parser')
    population_text_section = get_text_section('section1',soup)
    population = searchpattern('it\shas\s(\d.*?\d)\spopulation\s',population_text_section)
    population = refinestring({"$":"",",":"","%":""},population)
    population_dict[key] = population
population_dat = pd.DataFrame(list(population_dict.items()),columns =['city','population'] )

In [ ]:
population_dat['city'] = population_dat['city'].str.replace('-',' ')

In [ ]:
california_housing = california_housing.merge(population_dat, left_on='city',right_on='city')

Merging `area_total_km2`

In [23]:
area_dat = (pd.read_csv('https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/california_cities.csv',index_col=0)).loc[:,['city','area_total_km2']]

The dataset from jake vanderplas has city name with no spaces
so a quick replace should do the job so we are able to merge

In [24]:
california_housing['city'] = california_housing['city'].str.replace(" ","")

In [31]:
california_housing = california_housing.merge(area_dat,left_on='city',right_on='city',how='left')

In [32]:
california_housing.head()

,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,...,twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999,city,Latitude,Longitude,area_total_km2
0,8751.0,156.2,-3.7,50.7,49.3,151600.0,1059.0,1093.0,24.0,427.0,...,0.131,0.032,0.508,0.583,0.137,0.181,Adelanto,34.582769,-117.409214,145.107
1,7674.0,984.7,1.2,25.6,74.4,745000.0,2261.0,2488.0,23.0,928.0,...,0.064,0.899,0.038,0.000,0.962,0.000,AgouraHills,34.153339,-118.761675,20.260
2,32414.0,3104.1,0.2,53.0,47.0,729100.0,1607.0,2259.0,21.0,1006.0,...,0.261,0.633,0.019,0.119,0.667,0.036,Alameda,37.765206,-122.241636,59.465
3,7724.0,4319.4,-2.1,52.4,47.6,766000.0,1739.0,2501.0,21.0,991.0,...,0.138,0.821,0.012,0.084,0.849,0.000,Albany,37.886869,-122.297747,14.155
4,30990.0,4061.1,0.2,59.7,40.3,553800.0,1286.0,1629.0,22.0,1296.0,...,0.575,0.312,0.054,0.180,0.629,0.039,Alhambra,34.095286,-118.127014,19.766


In [ ]:
california_housing.to_csv('C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\california_housing.csv')

#  The End